In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_03")
display(data_dump)

# Lattice dynamics of a one-dimensional crystal 

Consider a one-dimensional crystal with lattice parameter $a$, so that the positions of atoms in the ideal lattice are $r_n=na$. Each atom may be displaced by a finite amount $u_n$ from the ideal position, so that the actual atomic positions are $r_n=na+u_n$.

<img src="figures/TBC-linear-chain.png" width="500" height="250" />

The energy can be computed in terms of a Taylor expansion: if $na$ are equilibrium lattice positions, $\partial E/\partial u_n = 0$ and so the lowest-order term involves the matrix of second derivatives,
$$
E(\mathbf{u}) \approx E_0 + \frac{1}{2} \sum_{ij} \left.\frac{\partial^2 E}{\partial u_i \partial u_j}\right|_{\mathbf{u}=0} u_i u_j
$$

The matrix of second derivatives $D_{ij} \equiv \left.{\partial^2 E}/{\partial u_i \partial u_j}\right|_{\mathbf{u}=0}$ is usually called the *matrix of force constants*, and defines the response of the crystal to perturbations of the atomic positions. 

## Exercise 1: harmonic energy and forces

The force acting on the atoms when they are displaced from their equilibrium position is given by $\mathbf{f}=-\partial E/\partial \mathbf{u}$. 

<span style="color:blue"> **01** Write down the expression in terms of $\mathbf{D}$ and $\mathbf{u}$. </span> 

In [ ]:
ex01_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

<span style="color:blue"> **02** Write a function that computes the energy and the forces for a lattice containing two atoms, with displacements $u_0$ and $u_1$. Check that the function gives the correct results by validating your input. </span>

In [ ]:
# set upt the code widget window
ex01_wci = WidgetCodeInput(
        function_name="lattice_energy_force", 
        function_parameters="D_00, D_01, D_10, D_11, u_0, u_1",
        docstring="""
Computes energy and force associated with a given displacement of two atoms in a harmonic lattice. 

:param D_ij: elements of the matrix of second derivatives
:param u_i: atomic displacements
        
:return: A tuple containing the lattice energy and the force, (E, f_0, f_1)
""",
        function_body="""

# Write your solution, and test it by moving the sliders

e = 0.0       # write the potential energy function here
f = [0, 0]    # writw the force function here
return e, f[0], f[1]
"""
        )

data_dump.register_field("ex02-function", ex01_wci, "function_body")

In [ ]:
def draw_forces(ax, D_00, D_01, D_10, D_11, u_0, u_1 ):
    ax.add_artist(plt.Circle((0+u_0, 0), 0.1, color='red'))
    ax.add_artist(plt.Circle((1+u_1, 0), 0.1, color='red'))
    
    ax.add_artist(plt.Line2D([-0.05,0.05], [-0.05, 0.05], color='k'))
    ax.add_artist(plt.Line2D([-0.05,0.05], [0.05, -0.05], color='k'))
    ax.add_artist(plt.Line2D([1-0.05,1+0.05], [-0.05, 0.05], color='k'))
    ax.add_artist(plt.Line2D([1-0.05,1+0.05], [0.05, -0.05], color='k'))
    
    U, f0, f1 = ex01_wci.get_function_object()(D_00, D_01, D_10, D_11, u_0, u_1)
        
    if f0**2>1e-6:
        arr0 = mpl.patches.FancyArrow(0+u_0,0,f0,0,width=0.02)
        ax.add_artist(arr0) 
    if f1**2>1e-6:
        arr1 = mpl.patches.FancyArrow(1+u_1,0,f1,0,width=0.02)
        ax.add_artist(arr1)    

    ax.set_xlim(-0.5,1.5)
    ax.set_xlabel("x/a")
    ax.set_ylim(-0.5,0.5) 
    ax.yaxis.set_ticks([])
    ax.set_aspect('equal')

In [ ]:
p = WidgetPlot(draw_forces, WidgetParbox(D_00 = (1.0, -2, 2, 0.1, r'$D_{00}$'),
                                         D_01 = (-1.0, -2, 2, 0.1, r'$D_{01}$'),
                                         D_10 = (-1.0, -2, 2, 0.1, r'$D_{10}$'),
                                         D_11 = (1.0, -2, 2, 0.1, r'$D_{11}$'),
                                         u_0 = (-0.1, -0.2, 0.2, 0.01, r'$u_0$'),
                                         u_1 = (0.05, -0.2, 0.2, 0.01, r'$u_0$'),
                                         ));

In [ ]:
ex01_wcc = WidgetCodeCheck(ex01_wci, ref_values = 
                           {
                               (0,0,0,0,0,0) : (0,0,0),                               
                               (1,2,3,4,0,0) : (0,0,0),
                               (0,0,0,0,1,2) : (0,0,0),
                               (1,2,3,4,1,2) : (13.5,-6,-10.5),
                               (1,-1,-1,1,1,1) : (0,0,0),
                               (1,1,1,1,1,1) : (2,-2,-2),
                               (1,0,0,1,1,-1) : (1, -1, 1)
                               
                           }, demo = p)

In [ ]:
display(ex01_wcc)

<span style="color:blue"> **03** Set the values of the matrix of force constants to large random values, and then adjust the displacement values so that both atoms have the same displacement $u_0=u_1>0$.    
What do you observe? Is this a physical behavior if these two atoms represent a cell in a periodic system?
</span>    

In [ ]:
ex03_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex03-answer", ex03_txt, "value")
display(ex03_txt)

## Properties of the matrix of force constants

When the matrix of force constants describes the interactions in a periodic crystal, it must fulfill several physical conditions:
* The matrix is symmetric $D_{ij}=D_{ji}$
* The elements only depend on the separation between the atoms, $D_{ij} \equiv -K_{|i-j|}$. $\mathbf{K}$ is called the matrix of force constants.
* The matrix must satisfy a condition called *acoustic sum rule*, $\sum_i D_{ij} = 0$

<span style="color:blue"> **04** Write the expression for the energy of a structure with two atoms, with a $\mathbf{D}$ matrix that fulfills the acoustic sum rule. What happens if the two atoms are displaced by the same amount?  
You can go back to the previous exercise, and enter a set of values consistent with the ASR and visualize the effect in practice. 
</span>    

In [ ]:
ex04_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex04-answer", ex04_txt, "value")
display(ex04_txt)

## A solution for a periodic lattice

We look for a way to express the time dependence of the lattice vibrations, $u_n(t)$. The lattice vibrations must satisfy Newton's equation, which implies that $m\ddot{u}_i=f_i = -\sum_j D_{ij} u_j$. 
To solve this in the most general way possible, we express $u_n(t)$ on a plane wave basis, 
$$
u_n(t) = \int \,d\omega dq \hat{u}(q,\omega) e^{\mathrm{i} (q n a - \omega t)}.
$$
Given the periodicity of the lattice, one only needs to consider $-\pi/a < q < \pi/a$.
By substituting into the equations of motion, and noting that both the time derivative and the sum over the matrix of force constants are linear operations that commute with the integration, one sees that the only way to ensure a consistent solution for any $\hat{u}(q,\omega)$ is to have
$$
m \omega^2 e^{\mathrm{i} (q i a - \omega t)} = \sum_j D_{ij} e^{\mathrm{i} (q j a - \omega t)},
$$
that is, there is a *dispersion relation* that links $q$ and $\omega$, $m \omega(q)^2 = -\sum_j K_{j} e^{-\mathrm{i} q j a}$.

## Exercise 2: the dispersion relation

<span style="color:blue"> **05** Write the function that computes $m \omega(q)^2$ given the force constants $K_{1\ldots 4}$, and plot the frequencies of the lattice vibrations as a function of the wavevector. You can check your solution by comparing the value of the function you computed (red line) with a reference implementation (dashed gray line) </span>

Hints:
* When you change the function, move the sliders to update the plot
* You will need [functions from numpy](https://numpy.org/doc/stable/reference/routines.math.html), that you can access with `np.XXXX` from the function body
* The sum extends to negative values of $j$. Ask yourself what should be the value of $K_{-j}$ given the properties of the matrix of force constants.  
* What happens if you combine the terms $j$ and $-j$? Is the dispersion relation real?
* Don't forget there is also a $K_0$! Use the acoustic sum rule to determine its value

In [ ]:
# set upt the code widget window
ex02_wci = WidgetCodeInput(
        function_name="m_omega_square", 
        function_parameters="K_1, K_2, K_3, K_4, q",
        docstring="""
Computes the dispersion relation for a one-dimensional lattice given the first four force constants

:param K_i: force constants
:param q: wavevector to compute the dispersion relation
        
:return: the value of m * omega**2
""",
        function_body="""
# Write your solution, and test it by moving the sliders
import numpy as np
return 0.0
"""
        )

data_dump.register_field("ex05-function", ex02_wci, "function_body")

In [ ]:
def reference_func_02(K_1, K_2, K_3, K_4, q):    
    K_0 = -2*(K_1+K_2)-K_3-K_4-K_4-K_3; return -(K_0 + 2*K_1 * np.cos(q*1)+2*K_2 * (2*np.cos(q)**2 - 1)
                                         +2*K_3 * np.cos(q*3)+2*K_4 * (np.cos(q)**4+np.sin(q)**4-6*np.cos(q)**2*np.sin(q)**2 ) )


In [ ]:
def plot_freq(ax, K_1, K_2, K_3, K_4):
    q = np.linspace(-np.pi, np.pi, 200)
    w2ref = reference_func_02( K_1, K_2, K_3, K_4, q)
        
    user_function = ex02_wci.get_function_object()         
    w2 = user_function(K_1, K_2, K_3, K_4, q)
        
    ax.plot(q, np.sqrt(np.abs(w2))*np.sign(w2ref), 'r', linewidth=2)
    ax.plot(q, np.sqrt(np.abs(w2ref))*np.sign(w2ref), 'k--')
        
    ax.set_xlim(-np.pi, np.pi)
    ax.set_xlabel("q a")
    ax.set_ylim(-1,4)
    ax.set_ylabel(r"$\omega\sqrt{m}$")


In [ ]:
a = WidgetPlot(plot_freq, WidgetParbox(K_1 = (1.0, -2, 2, 0.1, r'$K_{1}$'),
                                       K_2 = (0.0, -2, 2, 0.1, r'$K_{2}$'),
                                       K_3 = (0.0, -2, 2, 0.1, r'$K_{3}$'),
                                       K_4 = (0.0, -2, 2, 0.1, r'$K_{4}$'),
                                       ));

In [ ]:
ex02_wcc = WidgetCodeCheck(ex02_wci, ref_values = {
    x: reference_func_02(*x) for x in [ (1,2,3,4,0), (1,2,3,4,1), (1,1,1,1,0.5), (-1,1,-2,2,0.25) ]    
    }, demo = a)

In [ ]:
display(ex02_wcc)

Make sure the reference implementation matches your function for all values of the force constants. 
Observe what happens as you change the parameters. 

<span style="color:blue"> **06** Why does it make sense to truncate the expansion at $K_4$? Do you expect the force constants to increase or decrease as $j$ gets larger? Hint: go back to look at the expression of the force acting on atom $0$. How does it depend on the displacement of atom $j$? </span>

In [ ]:
ex06_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex06-answer", ex06_txt, "value")
display(ex06_txt)

Note that the plot visualizes $\operatorname{sign}(\omega^2) \sqrt{|m\omega^2|}$, so negative values on the plot correspond to so-called *imaginary frequencies*, i.e. values of $q$ for which $\omega^2$ is negative. Move around the sliders to get some negative lobes

<span style='color: blue'> **07** How will the energy change if you introduce a distortion with a periodicity corresponding to a $q$ associated with a negative $\omega^2$? What does this imply in terms of the stability of the crystal? </span>

In [ ]:
ex07_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex07-answer", ex07_txt, "value")
display(ex07_txt)